# Experiments MotionSense resampled to 30Hz

This notebook will perform basic experiments on the balanced MotionSense dataset with the following steps:
1. Quick load train, test and validation CSV subsets from the balanced MotionSense dataset using `PandasDatasetsIO` helper
2. Subclassing the `Dataset` interface using `PandasMultiModalDataset`
3. Apply the fourier transform on the dataset
4. Train and evaluate SVM, KNN and Random Forest classification models in both time and frequency domains

The experiments will evaluate the performance of SVM, KNN and RF models on the balanced MotionSense dataset in both time and frequency domains.

## Common imports and definitions

In [11]:
from pathlib import Path  # For defining dataset Paths
import sys                # For include librep package

# This must be done if librep is not installed via pip,
# as this directory (examples) is appart from librep package root
sys.path.append("..")

# Third party imports
import pandas as pd
import numpy as np

# Librep imports
from librep.utils.dataset import PandasDatasetsIO          # For quick load train, test and validation CSVs
from librep.datasets.multimodal import PandasMultiModalDataset # Wrap CSVs to librep's `Dataset` interface

## Loading data
Change the path to use in other datasets

In [12]:
# Path for MotionSense balanced view resampled to 30Hz with the same activities (and labels numbers)
# It is assumed that the directory will contain (train.csv, test.csv and validation.csv)
dataset_path = Path("../data/views/MotionSense/resampled_view_30Hz")

Once paths is defined, we can load the CSV as pandas dataframes

In [13]:
# MotionSense dataframes
train, validation, test = PandasDatasetsIO(dataset_path).load()

Letś take a look in the train dataframes

In [14]:
train.head()

,Unnamed: 0,userAcceleration.x-0,userAcceleration.x-1,userAcceleration.x-2,userAcceleration.x-3,userAcceleration.x-4,userAcceleration.x-5,userAcceleration.x-6,userAcceleration.x-7,userAcceleration.x-8,...,attitude.yaw-85,attitude.yaw-86,attitude.yaw-87,attitude.yaw-88,attitude.yaw-89,activity code,length,trial_code,index,user
0,0,-0.186833,-0.179195,-0.226435,-0.234763,-0.267824,-0.234534,-0.235421,-0.133759,-0.297125,...,2.156109,2.049595,1.995176,1.728849,1.746608,0,150,1,150,11
1,1,-0.054442,0.260099,0.022933,0.019339,0.148599,-0.036896,-0.125777,-0.110877,0.016260,...,-0.632965,-0.742283,-0.935655,-1.089888,-1.548738,0,150,1,900,12
2,2,-0.007696,-0.009515,0.051284,-0.082342,0.046316,0.062557,-0.032338,-0.108787,-0.090480,...,0.824134,1.034692,1.392084,1.444181,1.626350,0,150,1,1050,21
3,3,-0.435023,-0.557701,-0.284523,0.142448,0.545683,0.363495,0.006622,-0.042397,-0.412486,...,-3.207119,-2.682709,-3.187662,-2.674180,-3.534511,0,150,2,150,17
4,4,0.098066,0.398057,0.321284,-0.024039,0.372623,0.302234,0.199685,0.285311,0.319281,...,-0.294709,-0.208773,-0.335625,-0.209796,-0.416804,0,150,11,450,21


## Creating a Librep dataset from pandas dataframes

Change the features to use in other datasets

In [15]:
# MotionSense features to select
features = [
    "userAcceleration.x",
    "userAcceleration.y",
    "userAcceleration.z",
    "rotationRate.x",
    "rotationRate.y",
    "rotationRate.z"
]

# Creating the datasets

# Train
train_dataset = PandasMultiModalDataset(
    train,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

# Validation
validation_dataset = PandasMultiModalDataset(
    validation,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

# Test
test_dataset = PandasMultiModalDataset(
    test,
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)

## Inspect sample

In [6]:
# Lets print the first sample of kh_train dataset.
# Is a tuple, with an vector of 1800 elements as first element and the label as second
x = train_dataset[0]
print(x)

(array([-0.18683339, -0.17919536, -0.22643513, -0.23476344, -0.26782356,
       -0.23453445, -0.2354211 , -0.1337593 , -0.29712457, -0.31929832,
        0.00895068,  0.04136671, -0.44593038, -0.22804557,  0.05055849,
        0.18823314, -0.030559  , -0.07449108,  0.03599696, -0.05251227,
       -0.12043554, -0.29996206, -0.45056719,  0.14682301,  0.01642679,
        0.10086693,  0.15613708, -0.09381404, -0.27903132, -0.27132581,
       -0.19943854, -0.32235544, -0.28174771, -0.18833807, -0.17593017,
       -0.25836724, -0.41380716, -0.6389691 , -0.06662236,  0.61091861,
        0.12311657,  0.07509425,  0.13265687,  0.13606525, -0.0760303 ,
       -0.19980975,  0.00869801,  0.35475446, -0.23697373, -0.36777466,
       -0.24567402,  0.15071201,  0.14458744, -0.10843151, -0.0182671 ,
       -0.08241991, -0.03545809, -0.27308637, -0.31968743, -0.17252916,
        0.28114138,  0.32462763, -0.05580723,  0.13851364,  0.1309981 ,
       -0.13054706, -0.35103624, -0.20315897,  0.23055327,  0.1

In [7]:
# Inspecting sample
print(f"The sample 0: {x[0]}")
print(f"Shape of sample 0: {x[0].shape}")
print(f"The label of sample 0: {x[0]}")

The sample 0: [-0.18683339 -0.17919536 -0.22643513 -0.23476344 -0.26782356 -0.23453445
 -0.2354211  -0.1337593  -0.29712457 -0.31929832  0.00895068  0.04136671
 -0.44593038 -0.22804557  0.05055849  0.18823314 -0.030559   -0.07449108
  0.03599696 -0.05251227 -0.12043554 -0.29996206 -0.45056719  0.14682301
  0.01642679  0.10086693  0.15613708 -0.09381404 -0.27903132 -0.27132581
 -0.19943854 -0.32235544 -0.28174771 -0.18833807 -0.17593017 -0.25836724
 -0.41380716 -0.6389691  -0.06662236  0.61091861  0.12311657  0.07509425
  0.13265687  0.13606525 -0.0760303  -0.19980975  0.00869801  0.35475446
 -0.23697373 -0.36777466 -0.24567402  0.15071201  0.14458744 -0.10843151
 -0.0182671  -0.08241991 -0.03545809 -0.27308637 -0.31968743 -0.17252916
  0.28114138  0.32462763 -0.05580723  0.13851364  0.1309981  -0.13054706
 -0.35103624 -0.20315897  0.23055327  0.14958711 -0.21868229 -0.17026508
 -0.43630976 -0.14759122 -0.10772603  0.00561347 -0.03636226 -0.17544141
 -0.47363754 -0.32509977  0.07996614 

## Fourier Transform

In [8]:
from librep.datasets.multimodal import TransformMultiModalDataset
from librep.transforms.fft import FFT

In [9]:
fft_transform = FFT(centered = True)
transformer = TransformMultiModalDataset(transforms=[fft_transform], new_window_name_prefix="fft.")

### Use FFT in MotionSense

In [10]:
train_dataset_fft = transformer(train_dataset)
validation_dataset_fft = transformer(validation_dataset)
test_dataset_fft = transformer(test_dataset)

ValueError: invalid number of data points (0) specified

In [ ]:
train_dataset[:][0]

In [12]:
train_dataset_fft[:][0]

array([[ 0.4051    ,  1.02004708,  1.07645137, ...,  0.81610676,
         1.46963384,  0.38973439],
       [12.27775   ,  4.13506303,  2.19762137, ...,  2.24368189,
         3.28249292,  0.97427267],
       [ 9.830147  ,  4.4287738 ,  5.70037324, ...,  0.62088193,
         0.92075927,  0.48567778],
       ...,
       [ 1.303547  ,  4.4969612 ,  3.74262557, ...,  3.49895555,
         0.98426705,  7.3453344 ],
       [ 2.758783  ,  2.02576464,  0.89622788, ...,  3.34989765,
         2.32798646,  1.80279953],
       [ 9.720902  ,  1.55661245,  3.92530048, ...,  5.51747107,
         1.05656395,  2.57233132]])

## Train and evaluate Random Forest classifier

In [13]:
from librep.utils.workflow import SimpleTrainEvalWorkflow, MultiRunWorkflow
from librep.estimators import RandomForestClassifier
from librep.metrics.report import ClassificationReport
import yaml

reporter = ClassificationReport(use_accuracy=True, use_f1_score=True, use_classification_report=False, use_confusion_matrix=False, plot_confusion_matrix=False)
experiment = SimpleTrainEvalWorkflow(estimator=RandomForestClassifier, estimator_creation_kwags ={'n_estimators':100} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

In [14]:
combined_train_dset = PandasMultiModalDataset(
    pd.concat([train, validation]),
    feature_prefixes=features,
    label_columns="activity code",
    as_array=True
)


result = multi_run_experiment(combined_train_dset, test_dataset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058691.958451
    result:
    -   accuracy: 0.8274161735700197
        f1 score (macro): 0.8265047544211411
        f1 score (micro): 0.8274161735700197
        f1 score (weighted): 0.8283275927188983
    run id: 1
    start: 1662058685.637917
    time taken: 6.320533990859985
-   end: 1662058698.2649539
    result:
    -   accuracy: 0.814595660749507
        f1 score (macro): 0.8132132157540464
        f1 score (micro): 0.814595660749507
        f1 score (weighted): 0.8159781057449675
    run id: 2
    start: 1662058691.9584527
    time taken: 6.306501150131226
-   end: 1662058704.5732756
    result:
    -   accuracy: 0.8185404339250493
        f1 score (macro): 0.816785371802641
        f1 score (micro): 0.8185404339250493
        f1 score (weighted): 0.8202954960474576
    run id: 3
    start: 1662058698.2649558
    time taken: 6.308319807052612



In [15]:
combined_train_dset_fft = transformer(combined_train_dset)

result = multi_run_experiment(combined_train_dset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058709.0840907
    result:
    -   accuracy: 0.8224852071005917
        f1 score (macro): 0.8249361596193524
        f1 score (micro): 0.8224852071005917
        f1 score (weighted): 0.8200342545818312
    run id: 1
    start: 1662058704.8035216
    time taken: 4.280569076538086
-   end: 1662058713.3284512
    result:
    -   accuracy: 0.8284023668639053
        f1 score (macro): 0.8309968376624209
        f1 score (micro): 0.8284023668639053
        f1 score (weighted): 0.8258078960653898
    run id: 2
    start: 1662058709.0840921
    time taken: 4.244359016418457
-   end: 1662058717.5715759
    result:
    -   accuracy: 0.8175542406311637
        f1 score (macro): 0.8207150680020541
        f1 score (micro): 0.8175542406311638
        f1 score (weighted): 0.8143934132602735
    run id: 3
    start: 1662058713.3284528
    time taken: 4.2431230545043945



## Train and evaluate Support Vector Machine classifier

In [16]:
#from librep.estimators import SVC
from sklearn.svm import SVC

experiment = SimpleTrainEvalWorkflow(estimator=SVC, estimator_creation_kwags ={'C':3.0, 'kernel':"rbf"} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, test_dataset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058722.2264612
    result:
    -   accuracy: 0.6390532544378699
        f1 score (macro): 0.6133353886470502
        f1 score (micro): 0.6390532544378699
        f1 score (weighted): 0.6647711202286894
    run id: 1
    start: 1662058717.5770545
    time taken: 4.649406671524048
-   end: 1662058727.040984
    result:
    -   accuracy: 0.6390532544378699
        f1 score (macro): 0.6133353886470502
        f1 score (micro): 0.6390532544378699
        f1 score (weighted): 0.6647711202286894
    run id: 2
    start: 1662058722.226463
    time taken: 4.814520835876465
-   end: 1662058731.7785027
    result:
    -   accuracy: 0.6390532544378699
        f1 score (macro): 0.6133353886470502
        f1 score (micro): 0.6390532544378699
        f1 score (weighted): 0.6647711202286894
    run id: 3
    start: 1662058727.0409856
    time taken: 4.7375171184539795



In [17]:
result = multi_run_experiment(combined_train_dset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058732.6979492
    result:
    -   accuracy: 0.8648915187376726
        f1 score (macro): 0.864679230820583
        f1 score (micro): 0.8648915187376726
        f1 score (weighted): 0.8651038066547622
    run id: 1
    start: 1662058731.7835605
    time taken: 0.9143886566162109
-   end: 1662058733.6290216
    result:
    -   accuracy: 0.8648915187376726
        f1 score (macro): 0.864679230820583
        f1 score (micro): 0.8648915187376726
        f1 score (weighted): 0.8651038066547622
    run id: 2
    start: 1662058732.6979506
    time taken: 0.9310710430145264
-   end: 1662058734.5476532
    result:
    -   accuracy: 0.8648915187376726
        f1 score (macro): 0.864679230820583
        f1 score (micro): 0.8648915187376726
        f1 score (weighted): 0.8651038066547622
    run id: 3
    start: 1662058733.6290236
    time taken: 0.9186296463012695



## Train and evaluate K Neighbors Classifier classifier

In [18]:
#from librep.estimators import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

experiment = SimpleTrainEvalWorkflow(estimator=KNeighborsClassifier, estimator_creation_kwags ={'n_neighbors' :1} , do_not_instantiate=False, do_fit=True, evaluator=reporter)
multi_run_experiment = MultiRunWorkflow(workflow=experiment, num_runs=3, debug=False)

result = multi_run_experiment(combined_train_dset, test_dataset)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058734.8791718
    result:
    -   accuracy: 0.5029585798816568
        f1 score (macro): 0.49825668094091674
        f1 score (micro): 0.5029585798816568
        f1 score (weighted): 0.5076604788223971
    run id: 1
    start: 1662058734.5530987
    time taken: 0.32607316970825195
-   end: 1662058734.9548812
    result:
    -   accuracy: 0.5029585798816568
        f1 score (macro): 0.49825668094091674
        f1 score (micro): 0.5029585798816568
        f1 score (weighted): 0.5076604788223971
    run id: 2
    start: 1662058734.8791757
    time taken: 0.07570552825927734
-   end: 1662058735.0284004
    result:
    -   accuracy: 0.5029585798816568
        f1 score (macro): 0.49825668094091674
        f1 score (micro): 0.5029585798816568
        f1 score (weighted): 0.5076604788223971
    run id: 3
    start: 1662058734.9548829
    time taken: 0.0735175609588623



In [19]:
result = multi_run_experiment(combined_train_dset_fft, test_dataset_fft)
print(yaml.dump(result, sort_keys=True, indent=4))

runs:
-   end: 1662058735.080295
    result:
    -   accuracy: 0.7889546351084813
        f1 score (macro): 0.791422475633086
        f1 score (micro): 0.7889546351084813
        f1 score (weighted): 0.7864867945838767
    run id: 1
    start: 1662058735.0344865
    time taken: 0.04580855369567871
-   end: 1662058735.11995
    result:
    -   accuracy: 0.7889546351084813
        f1 score (macro): 0.791422475633086
        f1 score (micro): 0.7889546351084813
        f1 score (weighted): 0.7864867945838767
    run id: 2
    start: 1662058735.080297
    time taken: 0.03965306282043457
-   end: 1662058735.158503
    result:
    -   accuracy: 0.7889546351084813
        f1 score (macro): 0.791422475633086
        f1 score (micro): 0.7889546351084813
        f1 score (weighted): 0.7864867945838767
    run id: 3
    start: 1662058735.119952
    time taken: 0.03855109214782715

